In [1]:
# IMPORTAR LAS LIBRERIAS

import pandas as pd
from pysentimiento import create_analyzer
import re
from tqdm import tqdm
tqdm.pandas()

In [2]:
# COMENTARIOS BASURA O QUE NO AGREGAN VALOR

patron_basura = (
    r'\bninguna\b|'
    r'\bninguno\b|'
    r'ninguna observación|'
    r'ninguna observacion|'
    r'ninguna observación\.?|'
    r'ninguna observacion\.?|'
    r'sin comentario|'
    r'sin comentarios|'
    r'sin observación|'
    r'sin observacion|'
    r'sin observaciones|'
    r'\bno tengo\b|'
    r'no tengo comentarios|'
    r'no tengo ninguna observación|'
    r'no tengo observación|'
    r'no tengo observaciones|'
    r'no tengo queja|'
    r'no tengo queja alguna|'
    r'no tengo nada que decir|'
    r'no tengo nada más que decir|'
    r'no tengo nada mas que decir|'
    r'nada que decir|'
    r'nada para agregar|'
    r'nada que agregar|'
    r'nada más que decir|'
    r'nada mas que decir|'
    r'\bnada\b|'
    r'\bningún comentario\b|'
    r'\bningun comentario\b|'
    r'ningún aporte|'
    r'ningun aporte|'
    r'\bno aplica\b|'
    r'\bna\b|'
    r'\bn/a\b|'
    r'\bno responde\b|'
    r'\bno aplica\b|'
    r'\bok\b|'
    r'todo bien|'
    r'todo correcto|'
    r'todo normal|'
    r'\bgracias\b|'
    r'ningún comentario en especial|'
    r'ningun comentario en especial'
)

In [3]:
# ----------------- CARGA TU ARCHIVO -----------------
df = pd.read_excel(r"C:\Users\9 ----- SIG\Downloads\prueba_comentarios.xlsx")

df = df[['Id_profe', 'nombre_para', 'respuesta']].copy()
df['respuesta'] = df['respuesta'].astype(str).str.strip() # eliminar espacios
df = df[~df['respuesta'].str.contains(r'<[^>]+>', regex=True, na=False)]
df = df[df['respuesta'].str.len() > 15]  # filtrar respuestas muy cortas o vacías
df = df[~df['respuesta'].str.fullmatch(r'\.+')] # filtrar respuestas con solo puntos
df = df[~df['respuesta'].str.lower().str.contains('sin comentario|ninguno|n/a|no responde|Ninguna observación')]
df = df[~df['respuesta'].str.lower().str.contains(patron_basura, regex=True)] # filtrar respuestas basura
print(f"Comentarios válidos para analizar: {len(df)}")

Comentarios válidos para analizar: 24795


In [4]:
# ENTRENAMIENTO DEL MODELO

analyzer = create_analyzer(task="sentiment", lang="es")

# Lista de palabras/frases claramente negativas (para forzar detección)
palabras_muy_negativas = [
    "peor", "pesimo", "pésimo", "horrible", "terrible", "asco", "nunca viene", "abandona",
    "no explica", "insulta", "grita", "humilla", "racista", "grosero", "ridiculiza",
    "no hace nada", "perdida de tiempo", "perdida de tiempo", "no recomiendo", "mala persona"
]

def extraer_critica_inteligente(texto, prob_neg):
    texto_low = texto.lower()
    
    # 1. Si tiene más de 40% negativo → TODO el comentario es la crítica
    if prob_neg > 0.40:
        return texto.strip()[:300] + ("..." if len(texto)>300 else "")
    
    # 2. Si contiene alguna palabra muy negativa → extraer frase alrededor
    for palabra in palabras_muy_negativas:
        if palabra in texto_low:
            inicio = max(0, texto_low.find(palabra) - 40)
            fin = min(len(texto), texto_low.find(palabra) + len(palabra) + 80)
            return texto[inicio:fin].strip() + ("..." if fin < len(texto) else "")
    
    # 3. Conectores clásicos (pero, aunque, ojalá…)
    for conector in ["pero ", "aunque ", "ojalá ", "sin embargo ", "lo único malo ", "lastima que "]:
        if conector in texto_low:
            parte = texto_low.split(conector, 1)[1]
            parte = parte.split('.')[0].split(' y ')[0].split(', pero')[0]
            return parte.strip().capitalize()
    
    return "Ninguno detectado"

Identifica y extrae automáticamente la parte más crítica o negativa de un comentario usando la probabilidad de negatividad, palabras muy negativas y conectores típicos de crítica.

In [5]:
# APLICACION DEL MODELO

print("Analizando cada comentario con detección inteligente de críticas...")

def analizar(row):
    texto = row['respuesta']
    pred = analyzer.predict(texto)
    
    pos = pred.probas["POS"]
    neg = pred.probas["NEG"]
    neu = pred.probas["NEU"]
    total = pos + neg + neu + 1e-8
    
    porc_pos = round(100 * pos / total, 1)
    porc_neg = round(100 * neg / total, 1)
    porc_neu = round(100 * neu / total, 1)
    
    critica = extraer_critica_inteligente(texto, neg)
    
    return pd.Series({
        'Id_profe'       : row['Id_profe'],
        'Profesor'       : row['nombre_para'].strip().title(),
        'Comentario'     : texto,
        'Sentimiento'    : f"{porc_pos}% positivo — {porc_neg}% negativo — {porc_neu}% neutro",
        'Punto crítico'  : critica
    })

resultado = df.progress_apply(analizar, axis=1)

Analizando cada comentario con detección inteligente de críticas...


100%|██████████| 24795/24795 [1:37:49<00:00,  4.22it/s]  


Analiza cada comentario para calcular sus porcentajes de sentimiento y extraer su parte crítica, devolviendo un registro estructurado con profesor, comentario, sentimiento y punto crítico.

In [6]:
# CONSTRUIR TABLA FINAL

# Extraer con regex los valores numéricos antes del símbolo %
resultado['positivo'] = resultado['Sentimiento'].str.extract(r'(\d+\.?\d*)%\s*positivo')
resultado['negativo'] = resultado['Sentimiento'].str.extract(r'(\d+\.?\d*)%\s*negativo')
resultado['neutro']   = resultado['Sentimiento'].str.extract(r'(\d+\.?\d*)%\s*neutro')

# Convertir a float
resultado[['positivo','negativo','neutro']] = resultado[['positivo','negativo','neutro']].astype(float)
# Eliminar columna original
resultado = resultado.drop(columns=['Sentimiento'])

# Mover "Punto crítico" al final
col = [c for c in resultado.columns if c != 'Punto crítico'] + ['Punto crítico']
resultado = resultado[col]

resultado

,Id_profe,Profesor,Comentario,positivo,negativo,neutro,Punto crítico
9,103061,Rafael Toro,Muy buen profesor,95.9,0.5,3.6,Ninguno detectado
12,501023,Willian Gongora,"Aprendí mucho, buen profesor",95.2,0.5,4.4,Ninguno detectado
32,401547,German Florez,Aspectos por mejorar a la hora de hacer más in...,71.2,1.0,27.8,Bien
55,500062,Bethy Diaz,"La verdad la profe es cero empatíca, explica t...",0.3,98.1,1.6,"La verdad la profe es cero empatíca, explica t..."
58,501023,Willian Gongora,"Material de apoyo más didáctico, como por ejem...",82.6,0.7,16.7,Ninguno detectado
...,...,...,...,...,...,...,...
59228,301838,Willam Barreto,"Un excelente maestro, que buscò durante el tie...",97.3,0.4,2.3,Ninguno detectado
59235,501952,Machled Ramirez,muy buena profesora,96.2,0.4,3.4,Ninguno detectado
59237,501952,Machled Ramirez,muy buena profesora,96.2,0.4,3.4,Ninguno detectado
59239,502079,Nestor Penagos,Buena metodología de implementación de circuit...,85.0,0.7,14.3,Ninguno detectado


Separa los porcentajes de sentimiento en columnas numéricas, elimina la columna original y reorganiza la tabla dejando “Punto crítico” al final.

In [7]:
# AJUSTE FINAL DE NEUTROS

# --- PALABRAS POSITIVAS ---
palabras_positivas = [
    "buen", "buena", "excelente", "muy bueno", "muy buena",
    "gran profesor", "gran docente", "aprendí", "aprendi",
    "clase", "clases", "me gustó", "me gusto", "amable",
    "paciente", "claro", "explica bien", "responsable",
    "maravilloso", "maravillosa"
]

def contiene_palabra_positiva(texto):
    t = texto.lower()
    return any(p in t for p in palabras_positivas)


# --- AJUSTE FINAL ---
def ajustar_sentimiento(row):

    # 1. Punto crítico SIN problemas
    if row['Punto crítico'] != "Ninguno detectado":
        return row['positivo'], row['negativo'], row['neutro']

    # 2. Neutro menor al 60%
    if row['neutro'] >= 60:
        return row['positivo'], row['negativo'], row['neutro']

    # 3. El comentario contiene palabras positivas
    if not contiene_palabra_positiva(row['Comentario']):
        return row['positivo'], row['negativo'], row['neutro']

    # --- LAS 3 CONDICIONES SE CUMPLIERON →
    # convertir neutro en positivo
    nuevo_positivo = row['positivo'] + row['neutro']
    nuevo_negativo = row['negativo']
    nuevo_neutro = 0  # reflejar el ajuste

    return nuevo_positivo, nuevo_negativo, nuevo_neutro


# Aplicar la función fila por fila al DataFrame RESULTADO
resultado[['positivo', 'negativo', 'neutro']] = resultado.apply(
    lambda fila: ajustar_sentimiento(fila),
    axis=1,
    result_type='expand'
)

Ajusta los porcentajes de sentimiento convirtiendo el neutro en positivo solo cuando no hay crítica, el neutro es bajo y el comentario contiene palabras positivas.

In [8]:
# VER TABLA FINAL 

resultado.head(2)

,Id_profe,Profesor,Comentario,positivo,negativo,neutro,Punto crítico
9,103061,Rafael Toro,Muy buen profesor,99.5,0.5,0.0,Ninguno detectado
12,501023,Willian Gongora,"Aprendí mucho, buen profesor",99.6,0.5,0.0,Ninguno detectado


In [10]:
# CALCULAR INDICADOR

resultado['Indicador'] = round(5 * (resultado['positivo'] / (resultado['positivo'] + resultado['negativo'])),2)
resultado['Profesor'] = resultado['Profesor'].str.upper()
resultado.head(2)

,Id_profe,Profesor,Comentario,positivo,negativo,neutro,Punto crítico,Indicador
9,103061,RAFAEL TORO,Muy buen profesor,99.5,0.5,0.0,Ninguno detectado,4.97
12,501023,WILLIAN GONGORA,"Aprendí mucho, buen profesor",99.6,0.5,0.0,Ninguno detectado,4.98


In [11]:
# GUARDAR EXCEL

resultado.to_excel(r"C:\Users\9 ----- SIG\Documents\EVALUACION DOCENTE\analisis_de_sentimiento.xlsx", index=False)